# Relation extraction using distant supervision: experiments

In [7]:
__author__ = "Bill MacCartney and Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [Set-up](#Set-up)
1. [Building a classifier](#Building-a-classifier)
  1. [Featurizers](#Featurizers)
  1. [Experiments](#Experiments)
1. [Analysis](#Analysis)
  1. [Examining the trained models](#Examining-the-trained-models)
  1. [Discovering new relation instances](#Discovering-new-relation-instances)

## Overview

OK, it's time to get (halfway) serious. Let's apply real machine learning to train a classifier on the training data, and see how it performs on the test data. We'll begin with one of the simplest machine learning setups: a bag-of-words feature representation, and a linear model trained using logistic regression.

Just like we did in the unit on [supervised sentiment analysis](https://github.com/cgpotts/cs224u/blob/master/sst_02_hand_built_features.ipynb), we'll leverage the `sklearn` library, and we'll introduce functions for featurizing instances, training models, making predictions, and evaluating results.

## Set-up

See [the first notebook in this unit](rel_ext_01_task.ipynb#Set-up) for set-up instructions.

In [8]:
from collections import Counter
import os
import rel_ext
import utils

In [9]:
# Set all the random seeds for reproducibility. Only the
# system seed is relevant for this notebook.

utils.fix_random_seeds()

In [10]:
try:
    # python package (nlp) location - two levels up from this file
    src_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
    # add package to sys.path if it's not already there
    if src_path not in sys.path:
        sys.path.extend([src_path])
except NameError:
    print('issue with adding to path, probably due to __file__ not being defined')
    src_path = None

In [11]:
rel_ext_data_home = os.path.join(src_path,'data')
rel_ext_data_home

'/Users/timmaecker/Google Drive/Career and Skills/Learning/MSc Machine Learning - UCL/2 COMP0087 Statistical Natural Language Processing/Coursework/nlpproject/nlp/data'

With the following steps, we build up the dataset we'll use for experiments; it unites a corpus and a knowledge base in the way we described in [the previous notebook](rel_ext_01_task.ipynb).

In [12]:
corpus = rel_ext.Corpus(os.path.join(rel_ext_data_home, 'example_inputs_long_names_pos_and_neg.tsv.gz'))

print('Read {0:,} examples'.format(len(corpus)))

Read 86,298 examples


In [13]:
kb = rel_ext.KB(os.path.join(rel_ext_data_home, 'example_kb.tsv.gz'))

print('Read {0:,} KB triples'.format(len(kb)))

Read 21,458 KB triples


In [14]:
dataset = rel_ext.Dataset(corpus, kb)

The following code splits up our data in a way that supports experimentation:

In [15]:
splits = dataset.build_splits()

splits

{'tiny': Corpus with 4,151 examples; KB with 244 triples,
 'train': Corpus with 61,481 examples; KB with 15,632 triples,
 'dev': Corpus with 20,666 examples; KB with 5,582 triples,
 'all': Corpus with 86,298 examples; KB with 21,458 triples}

## Building a classifier

### Featurizers

Featurizers are functions which define the feature representation for our model. The primary input to a featurizer will be the `KBTriple` for which we are generating features. But since our features will be derived from corpus examples containing the entities of the `KBTriple`, we must also pass in a reference to a `Corpus`. And in order to make it easy to combine different featurizers, we'll also pass in a feature counter to hold the results.

Here's an implementation for a very simple bag-of-words featurizer. It finds all the corpus examples containing the two entities in the `KBTriple`, breaks the phrase appearing between the two entity mentions into words, and counts the words. Note that it makes no distinction between "forward" and "reverse" examples.

In [16]:
def simple_bag_of_words_featurizer(kbt, corpus, feature_counter):
    for ex in corpus.get_examples_for_entities(kbt.sbj, kbt.obj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    for ex in corpus.get_examples_for_entities(kbt.obj, kbt.sbj):
        for word in ex.middle.split(' '):
            feature_counter[word] += 1
    return feature_counter

Here's how this featurizer works on a single example:

In [17]:
kbt = kb.kb_triples[2]

kbt

KBTriple(rel='supplier of', sbj='Iri Group Holdings Inc', obj='DS Smith PLC')

In [18]:
kbt.sbj

'Iri Group Holdings Inc'

In [19]:
kbt.obj

'DS Smith PLC'

In [20]:
corpus.get_examples_for_entities(kbt.sbj, kbt.obj)

[]

In [21]:
#Find for which triplets we have examples
for kbt in kb.kb_triples:
    if len(corpus.get_examples_for_entities(kbt.sbj, kbt.obj))>0:
        print(kbt.sbj, kbt.obj)
        print(corpus.get_examples_for_entities(kbt.sbj, kbt.obj)[0].middle)
        break
    

Sprint Corp Brightstar Corp
 and SoftBank, is likely to accelerate under new chief executive Marcelo Claure, plucked from handset reseller 


In [22]:
corpus.get_examples_for_entities(kbt.sbj, kbt.obj)[0].middle

' and SoftBank, is likely to accelerate under new chief executive Marcelo Claure, plucked from handset reseller '

In [23]:
corpus.get_examples_for_entities(kbt.sbj, kbt.obj)[0]

Example(entity_1='Sprint Corp', entity_2='Brightstar Corp', left="Sprint's aggressive strategy on new handsets and price cuts, with the unveiling last week of a smartphone developed by Sharp exclusive for ", mention_1='Sprint Corp', middle=' and SoftBank, is likely to accelerate under new chief executive Marcelo Claure, plucked from handset reseller ', mention_2='Brightstar Corp', right=' which SoftBank acquired last year.', left_POS="Sprint's aggressive strategy on new handsets and price cuts, with the unveiling last week of a smartphone developed by Sharp exclusive for ", mention_1_POS='Sprint Corp', middle_POS=' and SoftBank, is likely to accelerate under new chief executive Marcelo Claure, plucked from handset reseller ', mention_2_POS='Brightstar Corp', right_POS=' which SoftBank acquired last year.')

In [24]:
simple_bag_of_words_featurizer(kbt, corpus, Counter())

Counter({'': 8,
         'and': 4,
         'SoftBank,': 2,
         'is': 2,
         'likely': 2,
         'to': 2,
         'accelerate': 2,
         'under': 2,
         'new': 2,
         'chief': 2,
         'executive': 2,
         'Marcelo': 2,
         'Claure,': 2,
         'plucked': 2,
         'from': 2,
         'handset': 2,
         'reseller': 2,
         'which': 2,
         'SoftBank': 2,
         'acquired': 2,
         'last': 2,
         'year.SoftBank': 2,
         'CEO': 2,
         'founder': 2,
         'Masayoshi': 2,
         'Son': 2,
         'has': 2,
         'vowed': 2,
         'bold': 2,
         'moves': 2,
         'by': 2})

You can experiment with adding new kinds of features just by implementing additional featurizers, following `simple_bag_of_words_featurizer` as an example.

Now, in order to apply machine learning algorithms such as those provided by `sklearn`, we need a way to convert datasets of `KBTriple`s into feature matrices. The following steps achieve that: 

In [25]:
kbts_by_rel, labels_by_rel = dataset.build_dataset()

featurized = dataset.featurize(kbts_by_rel, featurizers=[simple_bag_of_words_featurizer])

### Experiments

Now we need some functions to train models, make predictions, and evaluate the results. We'll start with `train_models()`. This function takes as arguments a dictionary of data splits, a list of featurizers, the name of the split on which to train (by default, 'train'), and a model factory, which is a function which initializes an `sklearn` classifier (by default, a logistic regression classifier). It returns a dictionary holding the featurizers, the vectorizer that was used to generate the training matrix, and a dictionary holding the trained models, one per relation.

In [26]:
train_result = rel_ext.train_models(
    splits,
    featurizers=[simple_bag_of_words_featurizer])

/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Next comes `predict()`. This function takes as arguments a dictionary of data splits, the outputs of `train_models()`, and the name of the split for which to make predictions. It returns two parallel dictionaries: one holding the predictions (grouped by relation), the other holding the true labels (again, grouped by relation).

In [27]:
predictions, true_labels = rel_ext.predict(
    splits, train_result, split_name='dev')

Now `evaluate_predictions()`. This function takes as arguments the parallel dictionaries of predictions and true labels produced by `predict()`. It prints summary statistics for each relation, including precision, recall, and F<sub>0.5</sub>-score, and it returns the macro-averaged F<sub>0.5</sub>-score.

In [28]:
rel_ext.evaluate_predictions(predictions, true_labels)

relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
customer of               0.902      0.986      0.917       2557       2869
supplier of               0.922      0.977      0.932       3025       3337
------------------    ---------  ---------  ---------  ---------  ---------
macro-average             0.912      0.982      0.925       5582       6206


0.9248700840206208

Finally, we introduce `rel_ext.experiment()`, which basically chains together `rel_ext.train_models()`, `rel_ext.predict()`, and `rel_ext.evaluate_predictions()`. For convenience, this function returns the output of `rel_ext.train_models()` as its result.

Running `rel_ext.experiment()` in its default configuration will give us a baseline result for machine-learned models.

In [29]:
_ = rel_ext.experiment(
    splits,
    featurizers=[simple_bag_of_words_featurizer])

/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


relation              precision     recall    f-score    support       size
------------------    ---------  ---------  ---------  ---------  ---------
customer of               0.902      0.986      0.917       2557       2869
supplier of               0.922      0.977      0.932       3025       3337
------------------    ---------  ---------  ---------  ---------  ---------
macro-average             0.912      0.982      0.925       5582       6206


Considering how vanilla our model is, these results are quite surprisingly good! We see huge gains for every relation over our `top_k_middles_classifier` from [the previous notebook](rel_ext_01_task.ipynb#A-simple-baseline-model). This strong performance is a powerful testament to the effectiveness of even the simplest forms of machine learning.

But there is still much more we can do. To make further gains, we must not treat the model as a black box. We must open it up and get visibility into what it has learned, and more importantly, where it still falls down.

## Analysis

### Examining the trained models

One important way to gain understanding of our trained model is to inspect the model weights. What features are strong positive indicators for each relation, and what features are strong negative indicators?

In [30]:
vectorizer = train_result['vectorizer']

if vectorizer is None:
    print("Model weights can be examined only if the featurizers "
            "are based in dicts (i.e., if `vectorize=True`).")

feature_names = vectorizer.get_feature_names()


/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [31]:
for rel, model in train_result['models'].items():
    print('Highest and lowest feature weights for relation {}:\n'.format(rel))
    try:
        coefs = model.coef_.toarray()
    except AttributeError:
        coefs = model.coef_
    break

Highest and lowest feature weights for relation customer of:



In [32]:
len(coefs[0])

46556

In [33]:
k=20
for rel, model in train_result['models'].items():
    print('Highest and lowest feature weights for relation {}:\n'.format(rel))
    try:
        coefs = model.coef_.toarray()
    except AttributeError:
        coefs = model.coef_
    sorted_weights = sorted([(wgt, idx) for idx, wgt in enumerate(coefs[0])], reverse=True)
    for wgt, idx in sorted_weights[:k]:
        print('{:10.3f} {}'.format(wgt, feature_names[idx]))
    print('{:>10s} {}'.format('.....', '.....'))
    for wgt, idx in sorted_weights[-k:]:
        print('{:10.3f} {}'.format(wgt, feature_names[idx]))
    print()


Highest and lowest feature weights for relation customer of:

     0.273 's
     0.231 chips
     0.203 week
     0.202 "
     0.199 their
     0.194 chip
     0.191 have
     0.180 some
     0.163 China
     0.161 sales
     0.161 plant
     0.160 05-Feb-2020
     0.158 fall
     0.157 AMC
     0.155 New
     0.154 this
     0.149 mobile
     0.142 was
     0.135 General
     0.131 supplier
     ..... .....
    -0.163 group
    -0.163 than
    -0.164 helped
    -0.166 expectations
    -0.166 lost
    -0.168 Chief
    -0.169 operating
    -0.170 carriers
    -0.175 Release
    -0.179 Inc,
    -0.184 airline
    -0.189 Holdings
    -0.195 Qorvo
    -0.199 read
    -0.207 :
    -0.215 Ltd
    -0.230 Inc
    -0.237 Corp
    -0.241 down
    -0.382 firms

Highest and lowest feature weights for relation supplier of:

     0.442 launch
     0.405 's
     0.392 chip
     0.391 chips
     0.386 products
     0.384 this
     0.375 2019
     0.373 2019.
     0.371 their
     0.367 plant
     0.36

In [34]:
rel_ext.examine_model_weights(train_result)

Highest and lowest feature weights for relation customer of:

     0.273 's
     0.231 chips
     0.203 week
     ..... .....
    -0.237 Corp
    -0.241 down
    -0.382 firms

Highest and lowest feature weights for relation supplier of:

     0.442 launch
     0.405 's
     0.392 chip
     ..... .....
    -0.558 carriers
    -0.572 firms
    -0.691 Corp



By and large, the high-weight features for each relation are pretty intuitive — they are words that are used to express the relation in question. (The counter-intuitive results merit a bit of investigation!)

The low-weight features (that is, features with large negative weights) may be a bit harder to understand. In some cases, however, they can be interpreted as features which indicate some _other_ relation which is anti-correlated with the target relation. (As an example, "directed" is a negative indicator for the `author` relation.)

__Optional exercise:__ Investigate one of the counter-intuitive high-weight features. Find the training examples which caused the feature to be included. Given the training data, does it make sense that this feature is a good predictor for the target relation?

<!--
- SPOILER: Using `penalty='l1'` results in somewhat less intuitive feature weights, and about the same performance.
- SPOILER: Using `penalty='l1', C=0.1` results in much more intuitive feature weights, but much worse performance.
-->

### Discovering new relation instances

Another way to gain insight into our trained models is to use them to discover new relation instances that don't currently appear in the KB. In fact, this is the whole point of building a relation extraction system: to extend an existing KB (or build a new one) using knowledge extracted from natural language text at scale. Can the models we've trained do this effectively?

Because the goal is to discover new relation instances which are *true* but *absent from the KB*, we can't evaluate this capability automatically. But we can generate candidate KB triples and manually evaluate them for correctness.

To do this, we'll start from corpus examples containing pairs of entities which do not belong to any relation in the KB (earlier, we described these as "negative examples"). We'll then apply our trained models to each pair of entities, and sort the results by probability assigned by the model, in order to find the most likely new instances for each relation.

In [35]:
rel_ext.find_new_relation_instances(
    dataset,
    featurizers=[simple_bag_of_words_featurizer])

/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/Users/timmaecker/opt/miniconda3/envs/nlpwcw1_rec/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Highest probability examples for relation customer of:

     1.000 KBTriple(rel='customer of', sbj='Amazon.com Inc', obj='Microsoft Corp')
     1.000 KBTriple(rel='customer of', sbj='Microsoft Corp', obj='Amazon.com Inc')
     1.000 KBTriple(rel='customer of', sbj='Tenaga Nasional Bhd', obj='Petronas Dagangan Bhd')
     1.000 KBTriple(rel='customer of', sbj='Petronas Dagangan Bhd', obj='Tenaga Nasional Bhd')
     1.000 KBTriple(rel='customer of', sbj='Walmart Inc', obj='Amazon.com Inc')
     1.000 KBTriple(rel='customer of', sbj='Amazon.com Inc', obj='Walmart Inc')
     0.999 KBTriple(rel='customer of', sbj='Dell Inc', obj='General Electric Co')
     0.999 KBTriple(rel='customer of', sbj='General Electric Co', obj='Dell Inc')
     0.998 KBTriple(rel='customer of', sbj='Nokia Oyj', obj='Shell PLC')
     0.998 KBTriple(rel='customer of', sbj='Shell PLC', obj='Nokia Oyj')

Highest probability examples for relation supplier of:

     1.000 KBTriple(rel='supplier of', sbj='Maxis Bhd', obj='

There are actually some good discoveries here! The predictions for the `author` relation seem especially good. Of course, there are also plenty of bad results, and a few that are downright comical. We may hope that as we improve our models and optimize performance in our automatic evaluations, the results we observe in this manual evaluation improve as well.

__Optional exercise:__ Note that every time we predict that a given relation holds between entities `X` and `Y`, we also predict, with equal confidence, that it holds between `Y` and `X`. Why? How could we fix this?

\[ [top](#Relation-extraction-using-distant-supervision) \]